In [79]:
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline

from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF,IDF
from pyspark.ml.feature import StringIndexer


from pyspark.ml.classification import NaiveBayes

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.ml.tuning import ParamGridBuilder,CrossValidator

import pandas as pd

https://spark.apache.org/docs/3.1.2/

https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.ml.classification.NaiveBayes.html#naivebayes

https://spark.apache.org/docs/3.1.2/api/java/org/apache/spark/ml/classification/NaiveBayes.html



https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html#multiclassclassificationevaluator


https://spark.apache.org/docs/3.1.2/api/java/org/apache/spark/ml/evaluation/MulticlassClassificationEvaluator.html




In [2]:
os.environ['JAVA_HOME'] = '/opt/jdk' #Mostra aonde está o JDK

In [3]:
spark = SparkSession.builder.master("local[4]") \
                    .appName('ml-teste') \
                    .getOrCreate()

sc=spark.sparkContext

21/10/29 08:06:53 WARN Utils: Your hostname, abner-Lenovo-G40-70 resolves to a loopback address: 127.0.1.1; using 192.168.15.20 instead (on interface wlp2s0)
21/10/29 08:06:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/29 08:06:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark

In [5]:
caminho_df_test = 'data_test.csv'
caminho_df_training = 'data_training.csv'

df_test = spark.read.csv(caminho_df_test,header=True)
df_training = spark.read.csv(caminho_df_training, header=True)

In [6]:
label_stringIdx = StringIndexer(inputCol = "topico", outputCol = "label")
tokenization = Tokenizer(inputCol="texto", outputCol="palavras")
remover_stopword = StopWordsRemover(inputCol="palavras", outputCol="palavras_filtradas")
hashingTF = HashingTF(inputCol="palavras_filtradas", outputCol="tf_features")
idf = IDF(inputCol="tf_features", outputCol="tf_idf_features")

In [ ]:
nb = NaiveBayes(featuresCol='tf_idf_features', labelCol='label', smoothing=1)
pipelineNB = Pipeline(stages=[label_stringIdx, tokenization, remover_stopword, hashingTF, idf, nb])

nbModel = pipelineNB.fit(df_training) #treina o modelo nb_model

In [8]:
nbparamGrid = ParamGridBuilder() \
               .addGrid(nb.smoothing, [0.0, 0.2, 0.4]) \
               .build()

In [9]:
nbcv = CrossValidator(estimator = pipelineNB,
                      estimatorParamMaps = nbparamGrid,
                      evaluator = MulticlassClassificationEvaluator(predictionCol="prediction"),
                      numFolds = 3)

In [ ]:
nbcvModel = nbcv.fit(df_training)

In [11]:
df_test_nbcvModel = nbcvModel.transform(df_test)

In [63]:
df_predictionAndLabels = df_test_nbcvModel.select(['prediction', 'label'])

In [66]:
labels = df_predictionAndLabels.rdd.map(lambda x: x.label).distinct().collect() # transforma o df para rdd e para poder extrair criar uma lista de labels distintas

21/10/29 10:28:11 WARN DAGScheduler: Broadcasting large task binary with size 22.1 MiB


In [30]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label' )

In [ ]:
vec_fMeasureByLabel = []
qtdClasses = 0
metric_list=[]

for label in sorted(labels):
    precisionByLabel = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: label})
    recallByLabel = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: label})
    fMeasureByLabel = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: label})
    
    metric_tuple_one = (label, precisionByLabel, recallByLabel,fMeasureByLabel, None , None)
    
    metric_list.append(metric_tuple_one)
    
    vec_fMeasureByLabel.append(fMeasureByLabel)
    qtdClasses +=1 

accuracy = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "accuracy"})
macroF1 = (sum(vec_fMeasureByLabel))/qtdClasses

metric_tuple_two = (None,None,None,None, accuracy, macroF1)

metric_list.append(metric_tuple_two)

In [83]:
dt_metrics = pd.DataFrame(metric_list)
dt_metrics.columns=(['class', 'precision', 'recall', 'F1', 'accuracy', 'macroF1'])
dt_metrics.to_csv('metrics_naive_bayes.csv', index=False)


In [ ]:
spark.stop()